In [1]:
import os
import cv2
import numpy as np
import torch
from PIL import Image
from torchvision import transforms
from matplotlib import pyplot as plt
from tensorflow import keras, convert_to_tensor, reshape
from Lenet import LeNet
import pandas as pd

2023-03-10 14:55:20.753076: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 14:55:21.295870: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-10 14:55:21.295925: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-10 14:55:21.295930: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
label_dict = pd.read_csv('label_dict.csv')
label_dict_inv = pd.read_csv('label_dict_inv.csv')

In [100]:
def preprocess_keras(img):
#     cv2.imshow("input",img)
#     cv2.waitKey(0)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_inv = cv2.bitwise_not(gray)
    _, binary = cv2.threshold(gray_inv, 35, 255, cv2.THRESH_BINARY)
    
    # get the indices of the black pixels
    y, x = np.nonzero(binary)
    # get the maximum and minimum x and y coordinates
    max_x = np.max(x)
    min_x = np.min(x)
    max_y = np.max(y)
    min_y = np.min(y)
    nonzero_indices = np.nonzero(binary)
    zero_indices = np.where(binary == 0)
    darkened_gray = gray
#     darkened_gray = gray.astype('float32')
#     darkened_gray[zero_indices] = 229+0.1*darkened_gray[zero_indices]
#     darkened_gray[nonzero_indices] *= 0.1
#     darkened_gray = darkened_gray.astype('uint8')
    cropped = darkened_gray[min_y:max_y, min_x:max_x]
    # _, thresh_img = cv2.threshold(gray_roi, 155, 255, cv2.THRESH_BINARY)
    inverted_img = cv2.bitwise_not(cropped)
#     inverted_img = cropped
    aspect_ratio = inverted_img.shape[1] / inverted_img.shape[0]
    # resize the image while keeping the aspect ratio
    new_height = 20
    new_width = int(new_height * aspect_ratio)
    if new_width > new_height:
        new_width = 20
        aspect_ratio = inverted_img.shape[0] / inverted_img.shape[1]
        new_height = int(new_width * aspect_ratio)
    resized_img = cv2.resize(inverted_img, (new_width, new_height))
    # pad the resized image with white pixels to make it 28x28
    pad_width = (
        ((28 - new_height) // 2, (28 - new_height + 1) // 2),  # no padding on top and bottom
        ((28 - new_width) // 2, (28 - new_width + 1) // 2))  # pad equally on both sides to make the width 28
    padded_img = np.pad(resized_img, pad_width, mode='constant', constant_values=0)
    normalized_img = (cv2.resize(padded_img, (28, 28)))
    return keras_preprocessing(normalized_img)


def keras_preprocessing(img):
    RGB = 1
    Length = 28
    Height = 28
    img = img.reshape(1, Length, Height, RGB)
    img = img/255
    return img

In [39]:

def extract_characters(img_name, model):
    img = cv2.imread(img_name)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to get a binary image
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 9, 5)

    # Find contours in the binary image
    contours, hierarchy = cv2.findContours(thresh, cv2.CHAIN_APPROX_SIMPLE, cv2.CHAIN_APPROX_SIMPLE)
    fields_points_sorted = []
    # Loop through the contours
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        threshold_value = 200
        black_found = False
        character = img[max((y + 3), 0):y + h - 3, max((x + 3), 0):x + w - 3]
        # Iterate over all pixels and check if their intensity values are above the threshold
        for row in range(character.shape[0]):
            for col in range(character.shape[1]):
                pixel_intensity = character[row, col]
                if (pixel_intensity < threshold_value).any():
                    black_found = True
                    break
            if black_found:
                break
        # Get the bounding rectangle of the contour
        if 30 < w < 100 and 40 < h < 100 and black_found:
            fields_points_sorted.append([max((y + 3), 0), y + h - 3, max((x + 3), 0), x + w - 3])
            # cv2.imwrite("C:/Users/30698/Desktop/KULeuven/Capita Selecta/test.jpeg", character)
            # show_one_prediction(character,model)
            #cv2.rectangle(img, (x + 3, y + 3), (x + w - 3, y + h - 3), (0, 255, 0), 2)
    fields_points_sorted.sort(key=lambda r: r[2])
    characters_sorted = []
    for field_points in fields_points_sorted:
        characters_sorted.append(img[field_points[0]:field_points[1], field_points[2]:field_points[3]])

    # Display the image with bounding boxes
    #cv2.imshow('image', img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return characters_sorted

In [76]:
def predict_date(model, characters_for_recognition, labels):
    i=0
    correct_fields=0
    for char in labels:
        if char != '/':
            processed_input = preprocess_keras(characters_for_recognition[i])            
            pred_vector = model.predict(processed_input)
            print(f"Prediction Vector: {pred_vector}")
            argmax = str(np.argmax(pred_vector))
            pred = str(label_dict_inv[argmax].get(0))
            print(f"Prediction: {pred}")
            if pred == char:
                correct_fields+=1
        i += 1
    return correct_fields

def predict_preprocessed_img(model, img):
    pred_vector = model.predict(img)
    print(f"Prediction Vector: {pred_vector}")
    argmax = str(np.argmax(pred_vector))
    pred = str(label_dict_inv[argmax].get(0))
    print(f"Prediction: {pred}")

In [7]:
# model_digits = LeNet()
# model_digits.load_state_dict(torch.load('saved_models/digits_model_nn3'))
model_digits = keras.models.load_model('./alphanum_model')

print("Model successfully loaded")

# extract_characters('C:/Users/30698/Desktop/KULeuven/Capita Selecta/formes/fields/0001_DATE.jpg')
# dir_path = 'C:/Users/30698/Desktop/KULeuven/Capita Selecta/formes/fione
fields_dir_path = "./Forms 24022023/fields/"

# Define the allowed image extensions
img_extensions = ["DATE.jpg", "DATE.jpeg", "DATE.png"]

2023-03-10 14:55:44.205842: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-10 14:55:44.208865: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-10 14:55:44.209006: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-10 14:55:44.209613: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Model successfully loaded


In [9]:
tmnist_df = pd.read_csv("94_character_TMNIST.csv")
tmnist_df.head()

,names,labels,1,2,3,4,5,6,7,8,...,775,776,777,778,779,780,781,782,783,784
0,Salsa-Regular,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,MouseMemoirs-Regular,D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Creepster-Regular,f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SeoulNamsan-Light,/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,HachiMaruPop-Regular,F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
X = tmnist_df.iloc[:, 2:].astype('float32') 
y = tmnist_df[['labels']]
X.shape

(274093, 784)

In [27]:
X.loc[0]

1      0.0
2      0.0
3      0.0
4      0.0
5      0.0
      ... 
780    0.0
781    0.0
782    0.0
783    0.0
784    0.0
Name: 0, Length: 784, dtype: float32

In [28]:
print(f"X Shape before reshaping: {X.shape}")
Length, Height = 28,28
# N of images 28x28
X = np.reshape(X.values, (X.shape[0] ,Length, Height)) 
print(f"X Shape after reshaping: {X.shape}")

X Shape before reshaping: (274093, 784)
X Shape after reshaping: (274093, 28, 28)


In [29]:
print(f'Before reshape: {X.shape}')
RGB = 1  # In this case only one instead of 3 because we dont have Color images
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], RGB)

#normalizing the image data
X = X/255
print(f'After reshape: {X.shape}')

Before reshape: (274093, 28, 28)
After reshape: (274093, 28, 28, 1)


In [36]:
img = X[0]
img = img.reshape(1, 28, 28, 1)
img.shape

(1, 28, 28, 1)

In [37]:
pred_vector = model_digits.predict(img)
print(f"Prediction Vector: {pred_vector}")
argmax = str(np.argmax(pred_vector))
pred = str(label_dict_inv[argmax].get(0))
print(f"Prediction: {pred}")

2023-03-10 15:24:13.135651: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8600


1/1 [==============================] - 3s 3s/step
Prediction Vector: [[1.00000000e+00 0.00000000e+00 9.90772652e-17 0.00000000e+00
  9.51197357e-24 0.00000000e+00 0.00000000e+00 3.80125578e-29
  1.01081622e-21 5.54695929e-21 1.09565498e-34 3.14445658e-38
  0.00000000e+00 1.12746616e-14 0.00000000e+00 0.00000000e+00
  1.90002560e-25 0.00000000e+00 0.00000000e+00 9.45172192e-22
  0.00000000e+00 0.00000000e+00 2.44911753e-25 7.31145263e-28
  0.00000000e+00 0.00000000e+00 1.36102243e-28 3.26005932e-34
  1.49812968e-10 3.16914217e-31 3.75025338e-37 0.00000000e+00
  1.67285791e-16 5.69690222e-28 0.00000000e+00 1.16248860e-36
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 6.13162543e-14 0.00000000e+00 2.42195952e-26
  1.26470214e-15 0.00000000e+00 0.00000000e+00 9.82703502e-11
  0.00000000e+00 0.00000000e+00 6.96705458e-30 2.28980131e-29
  4.21820350e-26 1.50872296e-28 0.00000000e+00 6.30990686e-38
  0.00000000e+00 1.41961773e-30 9.39603263e-23 1.71757940e-16
 

2023-03-10 15:24:14.656909: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [101]:
for root, dirs, files in os.walk(fields_dir_path):
        # Iterate over all files in the current directory
        correct=0
        print(f"Found {len(files)} files in '{fields_dir_path}'")
        for file in files:
            # Check if the file has an image extension            
            if any(file.endswith(ext) for ext in img_extensions):
                print(f"Checking for {file}:")
                # Add the full path of the image to the list
                img_path = os.path.join(root, file)
                
                characters_of_date = extract_characters(img_path, model_digits)
                file_index = int(file[:4])
                correct += predict_date(model_digits, characters_of_date, true_labels[file_index-1][2])
        print("Accuracy:", correct/(44*8))

Found 220 files in './Forms 24022023/fields/'
Checking for 0017_DATE.jpg:
1/1 [==============================] - 0s 14ms/step
Prediction Vector: [[9.24868289e-07 4.18470858e-09 8.94103991e-08 6.65632394e-09
  2.34783758e-04 3.97403437e-06 3.16098984e-03 1.69908686e-03
  1.23498481e-04 1.77982695e-08 3.89382526e-10 6.77738626e-06
  5.63841677e-05 3.63207720e-02 7.14000544e-06 2.41757749e-04
  7.01562458e-05 2.92959268e-10 2.35336144e-02 1.94358939e-12
  2.33269439e-08 2.11972282e-14 2.48904550e-03 2.36142199e-07
  1.33738290e-13 7.29752064e-04 2.33474770e-03 6.73052654e-08
  1.79708528e-03 4.13804628e-05 4.94868091e-10 2.21975704e-17
  7.92717002e-03 2.88779844e-08 8.60530436e-01 2.09461715e-09
  1.44167707e-05 2.62664185e-13 2.90102747e-07 8.60315286e-09
  1.38375547e-03 3.18699094e-05 9.07201247e-05 1.85920903e-03
  1.05452235e-03 2.48393277e-03 4.17629672e-05 1.85620138e-06
  2.44647129e-08 4.37393467e-16 4.02310754e-08 4.17327257e-08
  5.92993200e-03 4.05926548e-04 1.89009719e-04 1.

1/1 [==============================] - 0s 14ms/step
Prediction Vector: [[2.09923563e-13 5.36890843e-23 2.12026130e-09 8.74744884e-13
  1.58194200e-06 8.34484044e-12 1.83555028e-06 6.03834474e-13
  1.54853464e-04 3.89494903e-12 2.45481067e-18 3.32097843e-05
  3.49462175e-06 6.47574461e-06 6.41445650e-14 7.18455749e-06
  2.74523529e-07 2.04076514e-05 7.97771281e-05 2.89288260e-10
  2.15929504e-06 4.93181711e-12 1.61690135e-02 1.52643857e-17
  1.51458952e-08 4.85305282e-15 4.73442584e-01 2.82608182e-12
  6.60013955e-09 5.67391398e-05 7.00714338e-07 2.24308544e-23
  2.35286308e-07 1.25018556e-08 1.80264525e-02 7.94264736e-07
  1.03106137e-08 7.54223994e-13 1.43993934e-13 4.83662936e-12
  1.02919381e-10 2.60782195e-03 1.88053843e-06 2.59501531e-09
  2.24143756e-13 3.97129552e-05 8.31384206e-09 1.02812774e-15
  1.21776393e-08 2.61014964e-21 2.56088151e-09 8.15989087e-22
  1.82321727e-10 1.20681161e-05 1.00570721e-14 1.32289176e-06
  3.14384145e-08 1.16394610e-08 2.62805287e-14 1.86669079e-15

1/1 [==============================] - 0s 14ms/step
Prediction Vector: [[1.1826535e-27 1.1176476e-36 2.2497561e-11 7.5512534e-11 6.7873950e-20
  9.0497059e-15 8.5207120e-08 2.5155696e-18 7.3313134e-20 4.9149347e-21
  0.0000000e+00 9.9964929e-01 3.8864023e-21 1.4356016e-11 7.1700050e-22
  3.7414776e-12 1.1986900e-14 9.8873292e-19 1.5246430e-12 2.2458337e-07
  1.2251972e-16 6.0504996e-37 2.4366020e-07 1.4996425e-32 1.4163101e-19
  1.9410077e-13 8.0990527e-12 9.9959769e-20 2.2641821e-18 3.3941831e-19
  1.2956892e-19 1.3830904e-33 2.2730703e-14 4.2048367e-19 1.8275663e-12
  1.6664303e-14 1.0105350e-10 8.8097765e-29 3.7500476e-14 1.7362614e-13
  3.5988144e-17 9.1236982e-14 2.1660628e-04 2.6099606e-12 1.0875277e-25
  1.3241624e-13 4.0239503e-14 1.1070208e-24 2.5686512e-18 1.9620304e-32
  1.1761307e-07 1.3248343e-32 7.1290715e-21 1.3209798e-04 2.8938837e-11
  1.7667028e-11 6.8901910e-13 2.5096243e-09 1.2066654e-19 5.0275427e-26
  2.5719178e-13 1.4540890e-20 7.6716567e-12 8.4277271e-15 1.47334

1/1 [==============================] - 0s 13ms/step
Prediction Vector: [[4.6627537e-07 3.2909385e-05 7.9058960e-13 1.3056912e-12 2.1659920e-19
  4.8619771e-12 3.1491287e-10 1.9579229e-08 4.4838624e-14 1.2693140e-21
  1.7095899e-06 7.8293949e-09 2.6116228e-16 4.9521566e-07 6.0755303e-12
  1.7137693e-17 2.0980322e-06 2.5154739e-06 2.7076091e-17 9.0085349e-17
  5.4611602e-17 7.0491012e-07 1.9623384e-08 7.0481813e-01 6.0852500e-08
  6.0803438e-15 3.2015862e-08 1.0059725e-11 1.0063497e-09 1.6658310e-08
  5.1025126e-09 1.0428846e-18 2.0719062e-09 5.2849902e-08 6.2456067e-08
  2.5807920e-09 2.6655414e-12 2.4867754e-13 5.0062206e-19 4.6268993e-15
  8.0330368e-19 1.0428516e-09 4.9981353e-17 1.1990810e-06 5.5817742e-12
  2.8527027e-12 1.1654569e-17 4.2835133e-10 3.1179094e-12 8.7024219e-14
  1.0464610e-11 2.3407498e-01 1.7714810e-14 7.4029202e-08 1.3488422e-17
  3.4058457e-17 2.1509366e-15 3.6246935e-14 2.7390600e-21 5.3994396e-21
  3.9767535e-16 9.5877480e-16 3.1420152e-20 6.8861067e-12 8.99058

1/1 [==============================] - 0s 12ms/step
Prediction Vector: [[9.14175859e-18 4.52446512e-32 6.94754547e-08 1.36765692e-07
  1.60792044e-14 1.09189886e-13 1.23226198e-06 1.23822887e-15
  6.08327575e-16 1.34719553e-15 1.24019900e-32 9.95670080e-01
  2.15843467e-17 5.53581048e-09 1.70901894e-18 3.27431762e-11
  7.52891326e-12 9.61315125e-11 1.85620833e-15 2.02999599e-05
  2.19454124e-14 1.66784746e-22 1.32768342e-04 8.79483175e-25
  2.05770825e-13 3.08036374e-11 1.41089126e-10 2.30225906e-15
  3.21712774e-12 9.73232692e-12 4.91237097e-15 1.08340175e-24
  1.83233116e-11 4.37142128e-11 9.96089100e-10 4.16960043e-11
  6.83846660e-11 5.41551118e-24 1.06990581e-13 3.77673476e-10
  1.18692916e-15 5.70946346e-10 1.16408271e-04 2.56190874e-10
  8.25113566e-20 2.01062078e-09 3.69193426e-11 7.27857831e-19
  4.04194365e-13 7.77837426e-23 8.77012644e-06 1.50086786e-27
  6.56783926e-15 3.50999134e-03 1.17391776e-08 8.11134424e-11
  2.59796823e-10 7.65426265e-08 5.51507692e-14 1.32544655e-19

...

1/1 [==============================] - 0s 17ms/step
Prediction Vector: [[2.55358993e-12 2.57100126e-17 5.29106401e-05 3.99983546e-05
  2.63856333e-11 1.07818244e-06 2.38792245e-06 3.93188095e-08
  2.37222189e-08 9.94508446e-06 1.04588510e-22 7.83092558e-01
  1.01713781e-16 1.13678773e-08 2.74174520e-12 1.56080286e-08
  5.74811532e-10 6.91641727e-03 5.60073168e-11 1.06540543e-03
  1.34500730e-10 2.10169971e-16 1.20630148e-05 9.34148422e-15
  3.47069360e-08 2.53512367e-09 2.97155959e-04 3.06738792e-13
  1.05712162e-12 3.26711791e-09 3.21082522e-10 2.60380240e-22
  1.75011705e-09 8.70910344e-10 1.27627700e-10 4.73497930e-05
  1.03281786e-06 2.71683988e-13 4.67655326e-07 2.40383320e-03
  2.58159769e-08 5.94605553e-05 3.30601470e-05 4.04746658e-07
  3.75640545e-14 3.31546422e-11 4.20478305e-11 2.47838194e-09
  4.03152356e-08 1.38800244e-20 1.76371858e-02 2.02136975e-16
  1.06253973e-11 4.14466113e-02 6.35710720e-08 5.43832801e-09
  3.00432283e-08 3.03055253e-06 3.17834315e-06 9.22428366e-13

1/1 [==============================] - 0s 13ms/step
Prediction Vector: [[1.58625387e-11 2.24077470e-13 2.89102626e-08 2.03094842e-05
  4.32458548e-11 1.83836648e-12 5.21923749e-09 1.33236178e-16
  7.49173945e-10 1.65418772e-11 2.38434242e-16 3.52037810e-02
  5.16013159e-21 5.96457020e-13 2.13447142e-12 3.58264687e-13
  3.68952087e-08 1.99391792e-09 5.41971285e-27 1.14583783e-03
  9.05224923e-11 5.52128387e-09 7.12675003e-08 2.00940438e-08
  4.54655165e-05 2.15416551e-15 1.62552014e-10 1.73473666e-07
  3.15588389e-10 2.58752575e-10 2.09117101e-09 6.03219474e-10
  1.08191789e-09 5.70183145e-09 2.81181913e-18 4.64318406e-09
  4.29272227e-08 5.04845540e-11 4.53388344e-14 4.71632272e-01
  8.93569547e-20 2.62223576e-09 5.95241090e-07 1.36136620e-07
  1.50869474e-21 1.17011153e-12 8.41038834e-15 1.30162589e-10
  3.24027638e-10 2.49617632e-10 2.01198667e-01 1.36681699e-09
  3.25391976e-14 2.60989934e-01 3.56901331e-09 7.24520389e-15
  1.76915607e-10 1.08178995e-07 7.05939820e-11 1.04774037e-14

1/1 [==============================] - 0s 13ms/step
Prediction Vector: [[5.38447251e-16 1.08268505e-14 1.48192465e-11 1.78383318e-15
  1.83354241e-14 6.49799293e-14 4.83791021e-16 1.79655424e-09
  1.45525736e-13 1.08522078e-12 1.22930896e-05 2.01437069e-05
  7.62074015e-10 4.70946219e-07 7.80710349e-21 7.56098947e-04
  1.22687761e-05 6.67729413e-11 1.88595328e-13 1.71460446e-09
  2.59459202e-14 1.44975681e-11 8.81254618e-06 3.40242856e-08
  9.91010930e-14 7.14674586e-17 1.17403333e-05 4.62160637e-13
  2.22775132e-09 9.19022858e-10 3.94694924e-14 7.98095459e-25
  1.33946989e-04 1.65404379e-13 1.23047021e-05 1.33664162e-14
  1.08009048e-21 1.25459954e-08 2.60761224e-20 7.64193456e-17
  2.69123596e-22 5.81434776e-08 5.63696801e-14 4.09648046e-15
  3.51808101e-12 4.39413644e-11 3.83098086e-06 4.84445492e-13
  3.51458012e-12 2.33487051e-22 5.32373065e-07 2.71920952e-09
  5.25738002e-14 9.31802879e-06 2.21225939e-19 1.37269102e-11
  4.77702156e-10 1.57573688e-13 2.46478440e-18 1.04328091e-11

1/1 [==============================] - 0s 12ms/step
Prediction Vector: [[1.04250146e-08 8.36609021e-15 3.39993766e-09 3.37518269e-14
  9.01253811e-12 4.54565541e-09 3.79460507e-09 2.14438356e-09
  2.93603922e-12 2.39512793e-12 1.22722105e-24 7.17510673e-09
  4.61036725e-05 1.43412926e-09 2.79796612e-13 1.04874416e-11
  6.20331952e-10 2.21896644e-05 2.86040245e-06 2.47611176e-08
  1.46473263e-12 5.38851230e-09 4.30275694e-12 1.80334145e-14
  2.36804638e-08 2.55147747e-09 4.06533777e-20 7.72399159e-12
  3.45426444e-12 4.95778886e-06 7.45855721e-12 1.51402557e-21
  7.67856667e-10 5.12540277e-09 4.66653799e-12 1.16995817e-08
  6.17157325e-10 2.92380806e-22 7.07796408e-12 8.15253073e-12
  3.85050214e-10 2.72240347e-12 6.20098244e-12 3.73738089e-08
  4.90588043e-03 2.59479225e-06 9.07671247e-12 7.78498793e-12
  5.19221771e-07 8.90482760e-18 1.29020183e-10 5.02855626e-18
  5.17911045e-04 1.22220865e-08 7.65175007e-11 5.37200450e-15
  4.48510327e-05 8.30076494e-11 1.06931814e-12 2.16851470e-09

1/1 [==============================] - 0s 12ms/step
Prediction Vector: [[6.3531300e-14 8.3939212e-15 4.0496154e-08 2.0317580e-03 2.1276711e-10
  1.8029975e-11 4.6120554e-06 6.0066340e-13 1.1998894e-06 7.3810607e-09
  1.8949701e-16 1.3915857e-02 4.1546164e-20 8.4366697e-10 1.8306940e-07
  5.5674111e-11 4.1091216e-08 2.6852216e-09 4.4648493e-17 9.6881449e-06
  1.0685047e-07 1.6977905e-10 1.1154768e-04 1.0030715e-11 5.0768153e-07
  4.4994714e-10 4.8793561e-07 3.7240191e-06 2.3385574e-08 5.7963978e-10
  8.4245735e-11 2.6922933e-13 1.4197596e-07 1.0271626e-14 1.1944295e-12
  5.7035088e-10 1.2733114e-05 1.8029788e-09 7.8129421e-12 5.6106126e-01
  1.1564279e-11 2.3989655e-06 7.9124511e-05 1.7898705e-06 4.2460299e-17
  5.2062291e-12 9.1197700e-12 6.5406514e-10 1.0984661e-09 5.9944102e-13
  1.8296218e-01 5.3044358e-13 5.3215880e-11 2.2658573e-01 2.5276231e-07
  5.8558941e-10 2.3358191e-09 1.1098136e-08 6.3677985e-10 1.3375126e-14
  7.1932314e-11 1.5875755e-10 9.3178778e-06 2.9582201e-09 8.50723

In [90]:
label_dict.loc[0]["2"]

15

In [96]:
cv2.destroyAllWindows()